## 필요한 라이브러리 import

In [34]:
from pykospacing import Spacing
import pandas as pd
from konlpy.tag import Okt
from hanspell import spell_checker
import numpy as np
import re

spacing = Spacing()
okt = Okt()

## data.csv 파일 읽기

In [2]:
df = pd.read_csv('pre-data/data.csv',encoding='cp949',
                 low_memory=False)
df = df[['title', '개월']]

In [3]:
df

,title,개월
0,조리원에서 유축한것도 직접 먹이시나요!?,0개월
1,혼합수유에서 분유수유만 하려면 어떻게 해야하나요?,0개월
2,신생아 밥 달라고할때마다 줘도되나요?도와주세요,0개월
3,"수술 3일차인데, 모유수유 내일부터 시작해도 될까요?",0개월
4,조리원퇴소.. 유축기 사야하나요?,0개월
...,...,...
13790,냉동실에 있던 모유 냉장실에~,NaN
13791,아기가 많이 못빨았을때 유축하는게 맞나요?,NaN
13792,가슴이 땅땅하고 너무 아픈데 어떻게 해야할까요??,NaN
13793,남양 ㅇㅏ이엠마더 먹이는분계시나용 ㅠㅠ?,NaN


## 불용어(stopwords) 제거

### 불용어 불러오기

In [4]:
stopwords_df=pd.read_csv('pre-data/stopwords.csv', encoding='utf-8',
                 low_memory=False)
stopwords_df

,nursing,korean
0,가,아
1,가라,휴
2,가지,아이구
3,각,아이쿠
4,간,아이고
...,...,...
1431,흑,NaN
1432,흑흑,NaN
1433,흔히,NaN
1434,흠뻑,NaN


### 불용어 제거 (nuring)

In [6]:
# nursing
len_df = len(df)
for i in range(len_df):
    for j in range(len(stopwords_df['nursing'])):
        df['title'][i] = re.sub(" "+stopwords_df.loc[j]["nursing"]+" ", " ", df['title'][i])
    if i%100==1:
        print(f"{i} of {len_df}")

1 of 13795
101 of 13795
201 of 13795
301 of 13795
401 of 13795
501 of 13795
601 of 13795
701 of 13795
801 of 13795
901 of 13795
1001 of 13795
1101 of 13795
1201 of 13795
1301 of 13795
1401 of 13795
1501 of 13795
1601 of 13795
1701 of 13795
1801 of 13795
1901 of 13795
2001 of 13795
2101 of 13795
2201 of 13795
2301 of 13795
2401 of 13795
2501 of 13795
2601 of 13795
2701 of 13795
2801 of 13795
2901 of 13795
3001 of 13795
3101 of 13795
3201 of 13795
3301 of 13795
3401 of 13795
3501 of 13795
3601 of 13795
3701 of 13795
3801 of 13795
3901 of 13795
4001 of 13795
4101 of 13795
4201 of 13795
4301 of 13795
4401 of 13795
4501 of 13795
4601 of 13795
4701 of 13795
4801 of 13795
4901 of 13795
5001 of 13795
5101 of 13795
5201 of 13795
5301 of 13795
5401 of 13795
5501 of 13795
5601 of 13795
5701 of 13795
5801 of 13795
5901 of 13795
6001 of 13795
6101 of 13795
6201 of 13795
6301 of 13795
6401 of 13795
6501 of 13795
6601 of 13795
6701 of 13795
6801 of 13795
6901 of 13795
7001 of 13795
7101 of 13795
7201

In [7]:
df

,title,개월
0,조리원에서 유축한것도 먹이시나요!?,0개월
1,혼합수유에서 분유수유만 하려면 어떻게 해야하나요?,0개월
2,신생아 밥 달라고할때마다 줘도되나요?도와주세요,0개월
3,"수술 3일차인데, 모유수유 내일부터 시작해도 될까요?",0개월
4,조리원퇴소.. 유축기 사야하나요?,0개월
...,...,...
13790,냉동실에 모유 냉장실에~,NaN
13791,아기가 많이 못빨았을때 유축하는게 맞나요?,NaN
13792,가슴이 땅땅하고 아픈데 어떻게 해야할까요??,NaN
13793,남양 ㅇㅏ이엠마더 먹이는분계시나용 ㅠㅠ?,NaN


### 불용어(nursing) 제거한 df를 저장

In [8]:
df.to_csv("post-data/nursing_stopwords_removed.csv", mode='w')

In [9]:
# df = pd.read_csv('post-data/nursing_stopwords_removed.csv',low_memory=False)

## 동의어처리 (before spell check)

In [10]:
syn_df = pd.read_csv('pre-data/synonym.csv')
syn_df

,word,synonym,전문가 의미
0,분유,NaN,NaN
1,수유,NaN,삭제
2,모유수유,NaN,NaN
3,몇개월,시간,NaN
4,단유,다유,NaN
...,...,...,...
747,특정,NaN,삭제
748,판단,NaN,삭제
749,해당,NaN,삭제
750,현명,NaN,삭제


In [11]:
for idx in range(len(syn_df['전문가 의미'])):
    if syn_df['전문가 의미'][idx]=="삭제":
        syn_df['synonym'][idx] = "" 

# synonym 행 중 값이 없는 행은 삭제
syn_df = syn_df[syn_df['synonym'].notna()] 
syn_df = syn_df[['word', 'synonym']] 

# index 재구성
len_syn = len(syn_df['synonym'])
syn_index = [i for i in range(len_syn)]
syn_df = syn_df.set_index(pd.Index(syn_index))  

In [12]:
syn_df

,word,synonym
0,수유,
1,몇개월,시간
2,단유,다유
3,유축수유,유축
4,양,양
...,...,...
546,특정,
547,판단,
548,해당,
549,현명,


In [13]:
len_df = len(df)
for i in range(len_df):
    for j in range(len(syn_df['synonym'])):
        df['title'][i] = re.sub(" "+syn_df.loc[j]["word"]+" ", " "+syn_df.loc[j]["synonym"]+" ", df['title'][i])
    if i%100==1:
        print(f"{i} of {len_df}")

1 of 13795
101 of 13795
201 of 13795
301 of 13795
401 of 13795
501 of 13795
601 of 13795
701 of 13795
801 of 13795
901 of 13795
1001 of 13795
1101 of 13795
1201 of 13795
1301 of 13795
1401 of 13795
1501 of 13795
1601 of 13795
1701 of 13795
1801 of 13795
1901 of 13795
2001 of 13795
2101 of 13795
2201 of 13795
2301 of 13795
2401 of 13795
2501 of 13795
2601 of 13795
2701 of 13795
2801 of 13795
2901 of 13795
3001 of 13795
3101 of 13795
3201 of 13795
3301 of 13795
3401 of 13795
3501 of 13795
3601 of 13795
3701 of 13795
3801 of 13795
3901 of 13795
4001 of 13795
4101 of 13795
4201 of 13795
4301 of 13795
4401 of 13795
4501 of 13795
4601 of 13795
4701 of 13795
4801 of 13795
4901 of 13795
5001 of 13795
5101 of 13795
5201 of 13795
5301 of 13795
5401 of 13795
5501 of 13795
5601 of 13795
5701 of 13795
5801 of 13795
5901 of 13795
6001 of 13795
6101 of 13795
6201 of 13795
6301 of 13795
6401 of 13795
6501 of 13795
6601 of 13795
6701 of 13795
6801 of 13795
6901 of 13795
7001 of 13795
7101 of 13795
7201

In [14]:
df

,title,개월
0,조리원에서 유축한것도 먹이시나요!?,0개월
1,혼합수유에서 분유수유만 하려면 어떻게 해야하나요?,0개월
2,신생아 음식 달라고할때마다 줘도되나요?도와주세요,0개월
3,"수술 3일차인데, 모유수유 내일부터 시작해도 될까요?",0개월
4,조리원퇴소.. 유축 사야하나요?,0개월
...,...,...
13790,냉동실에 모유 냉장실에~,NaN
13791,아기가 많이 못빨았을때 유축하는게 맞나요?,NaN
13792,가슴이 땅땅하고 아픈데 어떻게 해야할까요??,NaN
13793,남양 ㅇㅏ이엠마더 먹이는분계시나용 ㅠㅠ?,NaN


### 동의어처리한 df 저장

In [15]:
df.to_csv("post-data/synyonym_checked_before_spell_chcking.csv", mode='w')

In [16]:
# df = pd.read_csv('post-data/synyonym_checked_before_spell_chcking.csv',low_memory=False)

## spell checking

In [17]:
spell_list = []

for i in range(len(df)):
# for i in range(10):
    spell_list.append(spacing(df['title'][i]))
    if i%100==1:
        print(i, spell_list[i])

df['spell'] = spell_list

df.to_csv("post-data/spell_checked.csv", mode='w')


2021-10-08 19:49:20.583862: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-08 19:49:20.584880: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3100220000 Hz


1 혼합수유에서 분유수유만 하려면 어떻게 해야 하나요?
101 출산 5일차 젖몸살ㅠ
201 모유가 안 나올 수도 있나요??
301 출산 12일차 젖양
401 유축기 필요한가요?
501 짧은 유두로도 ㅜㅜ 잘 먹겠죠?
601 조리원인데 애기가 잠들어요
701 초유 유축은 몇 시간이 텀이 적당할까요?
801 신생아 시간 봐주세요
901 20일차 신생아 배앓이? 변비? 분유 추천해주세요 ㅠㅠ
1001 신생아 분 유 갈아탈 때 섞어가면서 바꿔줘도 되나요?
1101 젖양 좀 봐주세요..ㅠ
1201 출산 4일차 모유 나오기 시작했는데요
1301 모유수유 음식 챙겨 먹어도 괜찮나요?
1401 하루 1000ml 남기면 안 된다?
1501 전혀 물지를 않아 요ㅠㅠ
1601 아기가 물어서 젖꼭지에서 피날 때
1701 모유수 유 너무 어려워 요ㅜ
1801 생후 30일 아기 분유량이요
1901 오늘 처음 모유 시작해서 질문이요(급해요 ㅠㅠ)
2001 어제 자연 출산 했는데요!
2101 이제 조리원에서 나오는 신생아가 분유 교체
2201 진짜 대 환장 파티네요..
2301 19일.아기에 요 직수만 하고 있고 유축은 안 해요. 다들 유축하시나요?
2401 모유사출이요
2501 젖몸살(?)
2601 생후 20일 전 신생아 분유 부탁드려 요~
2701 자연분만 3일차에 요 모유 시간 나올까요 ㅜㅜ?
2801 가슴이 찌릿찌릿 화끈거리며 아파요
2901 모유 몇 분 물려요..?
3001 15일 된 신생아 시간 지켜야 하나요?
3101 22일 신생아 분유 양 질문이요
3201 산후조리원에서 새벽에 모유짜서 어떻게 보관해요?
3301 생후 13일 15분 먹고 자는 데 반대편 유축해요?
3401 40일 된 5.7 킬로 아기 2시간마다 배 고프다고 울어요ㅜㅜ
3501 32일 차-처음보다 양이 줄어요 ㅠㅠ
3601 생후 35일 아기 분유
3701 분유 추천 도와 주세요~~!!
3801 36일 아가 분유량과 수 유텀 ㅠㅠ
3901 50일 아기 1시간 반마다 괜찮나요 ㅜㅜ??
4001 59일, 아기가 후

## remove stopwords(korean) after spell checking

In [18]:
kr_stopwords_df=stopwords_df.drop(['nursing'], axis=1).dropna()
kr_stopwords_df

,korean
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
670,일곱
671,여덟
672,아홉
673,령


In [19]:
# korean
len_df = len(df) 
for i in range(len_df):
    for j in range(len(kr_stopwords_df['korean'])):
        df['title'][i] = re.sub(" "+kr_stopwords_df.loc[j]["korean"]+" ", " ", df['title'][i])
    if i%100==1:
        print(f"{i} of {len_df}")

1 of 13795
101 of 13795
201 of 13795
301 of 13795
401 of 13795
501 of 13795
601 of 13795
701 of 13795
801 of 13795
901 of 13795
1001 of 13795
1101 of 13795
1201 of 13795
1301 of 13795
1401 of 13795
1501 of 13795
1601 of 13795
1701 of 13795
1801 of 13795
1901 of 13795
2001 of 13795
2101 of 13795
2201 of 13795
2301 of 13795
2401 of 13795
2501 of 13795
2601 of 13795
2701 of 13795
2801 of 13795
2901 of 13795
3001 of 13795
3101 of 13795
3201 of 13795
3301 of 13795
3401 of 13795
3501 of 13795
3601 of 13795
3701 of 13795
3801 of 13795
3901 of 13795
4001 of 13795
4101 of 13795
4201 of 13795
4301 of 13795
4401 of 13795
4501 of 13795
4601 of 13795
4701 of 13795
4801 of 13795
4901 of 13795
5001 of 13795
5101 of 13795
5201 of 13795
5301 of 13795
5401 of 13795
5501 of 13795
5601 of 13795
5701 of 13795
5801 of 13795
5901 of 13795
6001 of 13795
6101 of 13795
6201 of 13795
6301 of 13795
6401 of 13795
6501 of 13795
6601 of 13795
6701 of 13795
6801 of 13795
6901 of 13795
7001 of 13795
7101 of 13795
7201

In [20]:
df.to_csv("post-data/korean_stopwords_removed.csv", mode='w')

In [21]:
# df = pd.read_csv('post-data/korean_stopwords_removed.csv',low_memory=False)

## 동의어처리 (after spell check)

In [22]:
len_df = len(df)
for i in range(len_df):
    for j in range(len(syn_df['synonym'])):
        df['title'][i] = re.sub(" "+syn_df.loc[j]["word"]+" ", " "+syn_df.loc[j]["synonym"]+" ", df['title'][i])
    if i%100==1:
        print(f"{i} of {len_df}")

1 of 13795
101 of 13795
201 of 13795
301 of 13795
401 of 13795
501 of 13795
601 of 13795
701 of 13795
801 of 13795
901 of 13795
1001 of 13795
1101 of 13795
1201 of 13795
1301 of 13795
1401 of 13795
1501 of 13795
1601 of 13795
1701 of 13795
1801 of 13795
1901 of 13795
2001 of 13795
2101 of 13795
2201 of 13795
2301 of 13795
2401 of 13795
2501 of 13795
2601 of 13795
2701 of 13795
2801 of 13795
2901 of 13795
3001 of 13795
3101 of 13795
3201 of 13795
3301 of 13795
3401 of 13795
3501 of 13795
3601 of 13795
3701 of 13795
3801 of 13795
3901 of 13795
4001 of 13795
4101 of 13795
4201 of 13795
4301 of 13795
4401 of 13795
4501 of 13795
4601 of 13795
4701 of 13795
4801 of 13795
4901 of 13795
5001 of 13795
5101 of 13795
5201 of 13795
5301 of 13795
5401 of 13795
5501 of 13795
5601 of 13795
5701 of 13795
5801 of 13795
5901 of 13795
6001 of 13795
6101 of 13795
6201 of 13795
6301 of 13795
6401 of 13795
6501 of 13795
6601 of 13795
6701 of 13795
6801 of 13795
6901 of 13795
7001 of 13795
7101 of 13795
7201

### 동의어처리한 df 저장

In [23]:
df.to_csv("post-data/synyonym_checked_after_spell_chcking.csv", mode='w')

In [24]:
# df = pd.read_csv('post-data/synyonym_checked_after_spell_chcking.csv',low_memory=False)

## tokenization

In [25]:
df

,title,개월,spell
0,조리원에서 유축한것도 먹이시나요!?,0개월,조리원에서 유축한 것도 먹이시나요!?
1,혼합수유에서 분유수유만 하려면 해야하나요?,0개월,혼합수유에서 분유수유만 하려면 어떻게 해야 하나요?
2,신생아 음식 달라고할때마다 줘도되나요?도와주세요,0개월,신생아 음식 달라고 할 때마다 줘도 되나요?도와 주세요
3,"수술 3일차인데, 모유수유 내일부터 시작해도 될까요?",0개월,"수술 3일차인데, 모유수유 내 일부터 시작해도 될까요?"
4,조리원퇴소.. 유축 사야하나요?,0개월,조리원 퇴소.. 유축 사야 하나요?
...,...,...,...
13790,냉동실에 모유 냉장실에~,NaN,냉동실에 모유 냉장실에 ~
13791,아기가 많이 못빨았을때 유축하는게 맞나요?,NaN,아기가 많이 못 빨았을 때 유축하는 게 맞나요?
13792,가슴이 땅땅하고 아픈데 해야할까요??,NaN,가슴이 땅 땅하고 아픈데 어떻게 해야 할까요??
13793,남양 ㅇㅏ이엠마더 먹이는분계시나용 ㅠㅠ?,NaN,남양 ㅇㅏ이 엠마더 먹이는 분계시나 용 ㅠㅠ?


In [26]:
token_list=[]


def title_tokenization(df, idx):
    # return df['title'].replace([df['title'][idx]], " ".join(
    #     okt.nouns(df['title'][idx])))

    return " ".join(okt.nouns(df['spell'][idx]))

for i in range(len(df)):
    token_list.append(title_tokenization(df, i))
    if i%100==1:
        print(i, token_list[i])

df['token'] = token_list

df.to_csv("post-data/tokeniztioned.csv", mode='w')

1 혼합 수유 분유 수유 하나요
101 출산 젖 몸살
201 모유 안 수도
301 출산 젖
401 유축기
501 유두 도
601 조리 애기
701 초유 유축 몇 시간 텀 요
801 신생아 시간
901 신생아 배앓이 변비 분유 추천
1001 신생아 분 유 때
1101 젖 좀
1201 출산 모유 시작
1301 모유수유 음식
1401 하루 안
1501 전혀 요
1601 아기 물어 젖꼭지 피날 때
1701 유수 유 요
1801 생후 아기 분 유량
1901 오늘 처음 모유 시작 질문
2001 어제 자연 출산
2101 이제 조리 신생아 분유 교체
2201 진짜 환 파티
2301 아기 요 직 수만 유축 안 해 유 축하 시나
2401 모유 사출
2501 젖 몸살
2601 생후 전 신생아 분유 요
2701 자연 요 모유 시간 요
2801 가슴 화끈 거리
2901 모유 몇 분
3001 신생아 시간 하나요
3101 신생아 분유 양 질문
3201 산후조리원 새벽 모유 보관
3301 생후 데 반대편 유축해
3401 킬로 아기 배
3501 차 처음 양
3601 생후 아기 분유
3701 분유 추천 도
3801 아가 분 유량 수 유텀
3901 아기 반
4001 아기 후유 걸 어찌 나
4101 새벽 모유 수유 후 분유
4201 생후 아기 분유
4301 완모 중 아기 몸무게
4401 조언 좀 부탁
4501 수유 중 사 반사 몇 번
4601 여자 아가 분 유량 요
4701 직수 아기 몸무게 줄기 하나요
4801 생후 유량
4901 새벽 수유 울면 요
5001 아가 반
5101 아기 안 잠 자
5201 라면 바로 수유 요
5301 통 아기 하루 유량
5401 아가 분유 텀
5501 아가 갑자기 젖병 물 모유 실감 꼭지 요
5601 생후 아기 멕어
5701 개월 아기 땜 고민 요
5801 아가 분유 양
5901 아기 수유 이유 뭘
6001 분유
6101 백일 아기 수 유텀 분유 이요
6201 완분여
6301 아가 젖꼭지 쓰시
6401 아기 하루 몇 번
6501 아기
6601 밤중 수유
6701 아

In [27]:
df

,title,개월,spell,token
0,조리원에서 유축한것도 먹이시나요!?,0개월,조리원에서 유축한 것도 먹이시나요!?,조리 유축 것
1,혼합수유에서 분유수유만 하려면 해야하나요?,0개월,혼합수유에서 분유수유만 하려면 어떻게 해야 하나요?,혼합 수유 분유 수유 하나요
2,신생아 음식 달라고할때마다 줘도되나요?도와주세요,0개월,신생아 음식 달라고 할 때마다 줘도 되나요?도와 주세요,신생아 음식 달라 때 도
3,"수술 3일차인데, 모유수유 내일부터 시작해도 될까요?",0개월,"수술 3일차인데, 모유수유 내 일부터 시작해도 될까요?",수술 모유수유 내 일 시작 요
4,조리원퇴소.. 유축 사야하나요?,0개월,조리원 퇴소.. 유축 사야 하나요?,조리 퇴소 유축 하나요
...,...,...,...,...
13790,냉동실에 모유 냉장실에~,NaN,냉동실에 모유 냉장실에 ~,냉동 실 모유 냉장 실
13791,아기가 많이 못빨았을때 유축하는게 맞나요?,NaN,아기가 많이 못 빨았을 때 유축하는 게 맞나요?,아기 못 때 유 축하 게
13792,가슴이 땅땅하고 아픈데 해야할까요??,NaN,가슴이 땅 땅하고 아픈데 어떻게 해야 할까요??,가슴 땅 땅 요
13793,남양 ㅇㅏ이엠마더 먹이는분계시나용 ㅠㅠ?,NaN,남양 ㅇㅏ이 엠마더 먹이는 분계시나 용 ㅠㅠ?,남양 이 엠마 더 분계 시나 용


## making matrix

In [29]:
token_list = []
index_list = []

for i in range(len(df)):
    index_list.append(i)
    try:
        token_list += df['token'][i].split()
        if i%100==1:
            print(f"{i} of {len(df['token'])}")
    except AttributeError:  # token이 없는 경우 예외처리
        print(f"ERROR!\n{i} : {df['token'][i]}")

print(f"중복 제거 전 : {len(token_list)}개")
token_list = list(set(token_list))
print(f"중복 제거 후 : {len(token_list)}개")


1 of 13795
101 of 13795
201 of 13795
301 of 13795
401 of 13795
501 of 13795
601 of 13795
701 of 13795
801 of 13795
901 of 13795
1001 of 13795
1101 of 13795
1201 of 13795
1301 of 13795
1401 of 13795
1501 of 13795
1601 of 13795
1701 of 13795
1801 of 13795
1901 of 13795
2001 of 13795
2101 of 13795
2201 of 13795
2301 of 13795
2401 of 13795
2501 of 13795
2601 of 13795
2701 of 13795
2801 of 13795
2901 of 13795
3001 of 13795
3101 of 13795
3201 of 13795
3301 of 13795
3401 of 13795
3501 of 13795
3601 of 13795
3701 of 13795
3801 of 13795
3901 of 13795
4001 of 13795
4101 of 13795
4201 of 13795
4301 of 13795
4401 of 13795
4501 of 13795
4601 of 13795
4701 of 13795
4801 of 13795
4901 of 13795
5001 of 13795
5101 of 13795
5201 of 13795
5301 of 13795
5401 of 13795
5501 of 13795
5601 of 13795
5701 of 13795
5801 of 13795
5901 of 13795
6001 of 13795
6101 of 13795
6201 of 13795
6301 of 13795
6401 of 13795
6501 of 13795
6601 of 13795
6701 of 13795
6801 of 13795
6901 of 13795
7001 of 13795
7101 of 13795
7201

In [36]:
stopwords_list = list(np.array(stopwords_df['nursing'].tolist()))

print(f"stopwords 제거 전 : {len(token_list)}개")
token_list = list(set(token_list)-set(stopwords_list))
print(f"stopwords 제거 후 : {len(token_list)}개")

stopwords 제거 전 : 3499개
stopwords 제거 후 : 2344개


In [37]:
print("making matrix_df!")
matrix_df = pd.DataFrame(index=index_list, columns=token_list)

print("nan to 0")
matrix_df = matrix_df.fillna(0)

making matrix_df!
nan to 0


In [38]:
matrix_df

,생후,동산,수도,지남,량,오분,산부인과,확인,마더,노답,...,미달,빼기,막국수,벌써,절개,경제,플러스,려니,뚜껑,아떻
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13793,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
for col in range(len(matrix_df.columns)):
    print(f"{col} of {len(matrix_df.columns)} : {matrix_df.columns[col]}")
    for idx in matrix_df.index:
        try:
            cnt = df['token'][idx].split().count(matrix_df.columns[col])
            # print(matrix_df[matrix_df.columns[col]][idx], cnt)
            matrix_df[matrix_df.columns[col]][idx] = cnt
            
        except AttributeError:  # token이 없는 경우 예외처리
            pass

print("saving matrix_df!")
matrix_df.to_csv("post-data/matrix.csv", mode='w')

0 of 2344 : 생후
1 of 2344 : 동산
2 of 2344 : 수도
3 of 2344 : 지남
4 of 2344 : 량
5 of 2344 : 오분
6 of 2344 : 산부인과
7 of 2344 : 확인
8 of 2344 : 마더
9 of 2344 : 노답
10 of 2344 : 탈껀데
11 of 2344 : 고춧가루
12 of 2344 : 교
13 of 2344 : 마사지
14 of 2344 : 니
15 of 2344 : 궁금증
16 of 2344 : 신랑
17 of 2344 : 좀비
18 of 2344 : 상온
19 of 2344 : 건조
20 of 2344 : 엾는데
21 of 2344 : 중반
22 of 2344 : 축전
23 of 2344 : 다섯
24 of 2344 : 전자렌지
25 of 2344 : 유주시
26 of 2344 : 소화
27 of 2344 : 니플
28 of 2344 : 홍
29 of 2344 : 숟가락
30 of 2344 : 잠투정
31 of 2344 : 브레짜
32 of 2344 : 말랬는데
33 of 2344 : 바질
34 of 2344 : 비어
35 of 2344 : 안도
36 of 2344 : 젖비린내
37 of 2344 : 첨
38 of 2344 : 코로
39 of 2344 : 섭취
40 of 2344 : 류
41 of 2344 : 완모맘
42 of 2344 : 플라스틱
43 of 2344 : 이마트
44 of 2344 : 군것질
45 of 2344 : 정이
46 of 2344 : 채
47 of 2344 : 오늘
48 of 2344 : 두통
49 of 2344 : 개월수
50 of 2344 : 탈시
51 of 2344 : 힌
52 of 2344 : 접촉성
53 of 2344 : 티슈
54 of 2344 : 사람
55 of 2344 : 기절
56 of 2344 : 화장품
57 of 2344 : 딱지
58 of 2344 : 하렵니
59 of 2344 : 유제품
60 of 2344 : 크림
61 of 2344 : 쌤

484 of 2344 : 싱숭생숭
485 of 2344 : 색깔
486 of 2344 : 비교
487 of 2344 : 소시지
488 of 2344 : 짐
489 of 2344 : 남
490 of 2344 : 해외여행
491 of 2344 : 시점
492 of 2344 : 닥브
493 of 2344 : 카
494 of 2344 : 껀
495 of 2344 : 괜
496 of 2344 : 반나절
497 of 2344 : 상한
498 of 2344 : 기도
499 of 2344 : 관계
500 of 2344 : 뿜었는데
501 of 2344 : 텀좀
502 of 2344 : 타고
503 of 2344 : 예행
504 of 2344 : 가능
505 of 2344 : 생활
506 of 2344 : 상관관계
507 of 2344 : 일주
508 of 2344 : 젖집
509 of 2344 : 보틀
510 of 2344 : 유해
511 of 2344 : 뭉처
512 of 2344 : 유축량이
513 of 2344 : 케
514 of 2344 : 이뇨제
515 of 2344 : 노하우
516 of 2344 : 어뜨케
517 of 2344 : 초코파이
518 of 2344 : 머
519 of 2344 : 액
520 of 2344 : 유축팩
521 of 2344 : 방
522 of 2344 : 일간
523 of 2344 : 피해
524 of 2344 : 진입
525 of 2344 : 켜
526 of 2344 : 질의
527 of 2344 : 해외
528 of 2344 : 앞
529 of 2344 : 먹이
530 of 2344 : 비타민
531 of 2344 : 기념
532 of 2344 : 독해
533 of 2344 : 그저께
534 of 2344 : 언적힌
535 of 2344 : 직후
536 of 2344 : 어카
537 of 2344 : 관악구
538 of 2344 : 몸
539 of 2344 : 동탄
540 of 2344 : 작성
541 of 2344 : 전복
542 

962 of 2344 : 껏
963 of 2344 : 검진
964 of 2344 : 체함
965 of 2344 : 양수
966 of 2344 : 파
967 of 2344 : 긴급
968 of 2344 : 장기간
969 of 2344 : 원료
970 of 2344 : 세이프
971 of 2344 : 개운
972 of 2344 : 반영구
973 of 2344 : 간격
974 of 2344 : 어려움
975 of 2344 : 데용
976 of 2344 : 평편
977 of 2344 : 교체
978 of 2344 : 힙분워
979 of 2344 : 임페
980 of 2344 : 완분여
981 of 2344 : 밖
982 of 2344 : 어케넓히
983 of 2344 : 정보
984 of 2344 : 기저귀
985 of 2344 : 옹
986 of 2344 : 잠결
987 of 2344 : 소
988 of 2344 : 갈수록
989 of 2344 : 우린
990 of 2344 : 젖젖
991 of 2344 : 봐쥬세
992 of 2344 : 뿌
993 of 2344 : 관심
994 of 2344 : 내내
995 of 2344 : 소한
996 of 2344 : 뱃고레
997 of 2344 : 다음
998 of 2344 : 레이스
999 of 2344 : 스케쥴
1000 of 2344 : 이기
1001 of 2344 : 혈
1002 of 2344 : 고체
1003 of 2344 : 모유수유
1004 of 2344 : 애가
1005 of 2344 : 리파
1006 of 2344 : 아이엠
1007 of 2344 : 베바
1008 of 2344 : 촙촙
1009 of 2344 : 거부
1010 of 2344 : 통일
1011 of 2344 : 커서
1012 of 2344 : 어젠
1013 of 2344 : 소양증
1014 of 2344 : 햄버거
1015 of 2344 : 찜질
1016 of 2344 : 산통
1017 of 2344 : 카페
1018 of 2344 : 녹차


1416 of 2344 : 유한
1417 of 2344 : 지금
1418 of 2344 : 수포
1419 of 2344 : 압타미
1420 of 2344 : 불리
1421 of 2344 : 욧
1422 of 2344 : 육아
1423 of 2344 : 부분
1424 of 2344 : 때용
1425 of 2344 : 젖찰
1426 of 2344 : 취침
1427 of 2344 : 우려
1428 of 2344 : 텀되
1429 of 2344 : 일어나서
1430 of 2344 : 스테로이드
1431 of 2344 : 눠
1432 of 2344 : 증
1433 of 2344 : 백반
1434 of 2344 : 설사
1435 of 2344 : 돌맹이
1436 of 2344 : 삐뽀
1437 of 2344 : 열꽃
1438 of 2344 : 징얼
1439 of 2344 : 유가
1440 of 2344 : 방이
1441 of 2344 : 여러분
1442 of 2344 : 레모
1443 of 2344 : 유즙
1444 of 2344 : 기름
1445 of 2344 : 운오
1446 of 2344 : 혼돈
1447 of 2344 : 횟수
1448 of 2344 : 구내염
1449 of 2344 : 삼십분
1450 of 2344 : 유도기
1451 of 2344 : 켁
1452 of 2344 : 징조
1453 of 2344 : 느낌
1454 of 2344 : 사골
1455 of 2344 : 유축맘
1456 of 2344 : 유실
1457 of 2344 : 워
1458 of 2344 : 성숙
1459 of 2344 : 단맛
1460 of 2344 : 일과
1461 of 2344 : 사서
1462 of 2344 : 화
1463 of 2344 : 고인
1464 of 2344 : 중이
1465 of 2344 : 여야
1466 of 2344 : 액상뷴
1467 of 2344 : 충격
1468 of 2344 : 디스크
1469 of 2344 : 인생
1470 of 2344 : 틴
147

1869 of 2344 : 아인
1870 of 2344 : 주세영
1871 of 2344 : 달라
1872 of 2344 : 보쌈김치
1873 of 2344 : 완모중
1874 of 2344 : 루마
1875 of 2344 : 저체중
1876 of 2344 : 완분아
1877 of 2344 : 불규칙
1878 of 2344 : 미네랄
1879 of 2344 : 물어
1880 of 2344 : 유축기
1881 of 2344 : 좀
1882 of 2344 : 동해
1883 of 2344 : 핫
1884 of 2344 : 지구
1885 of 2344 : 돌진
1886 of 2344 : 모이
1887 of 2344 : 그람
1888 of 2344 : 과로
1889 of 2344 : 엉덩이
1890 of 2344 : 돈
1891 of 2344 : 청소기
1892 of 2344 : 런
1893 of 2344 : 쓰시
1894 of 2344 : 라떼
1895 of 2344 : 덩이
1896 of 2344 : 독박
1897 of 2344 : 교정
1898 of 2344 : 즘
1899 of 2344 : 주위
1900 of 2344 : 제도
1901 of 2344 : 개
1902 of 2344 : 킨
1903 of 2344 : 엉망
1904 of 2344 : 왓는데
1905 of 2344 : 뾰루
1906 of 2344 : 시간텀
1907 of 2344 : 오히려
1908 of 2344 : 유텀봐
1909 of 2344 : 제왕
1910 of 2344 : 어플
1911 of 2344 : 자세
1912 of 2344 : 사도
1913 of 2344 : 토텍
1914 of 2344 : 맞구왓는
1915 of 2344 : 쉬
1916 of 2344 : 부산
1917 of 2344 : 차이
1918 of 2344 : 일반
1919 of 2344 : 유땜
1920 of 2344 : 위액
1921 of 2344 : 성장
1922 of 2344 : 셧어
1923 of 2344 : 꿀팁
1

2321 of 2344 : 답
2322 of 2344 : 타임
2323 of 2344 : 칼디
2324 of 2344 : 지난
2325 of 2344 : 몽우리
2326 of 2344 : 헛구역질
2327 of 2344 : 재중
2328 of 2344 : 유저
2329 of 2344 : 횟
2330 of 2344 : 규칙
2331 of 2344 : 포기
2332 of 2344 : 혼유
2333 of 2344 : 벡스
2334 of 2344 : 미달
2335 of 2344 : 빼기
2336 of 2344 : 막국수
2337 of 2344 : 벌써
2338 of 2344 : 절개
2339 of 2344 : 경제
2340 of 2344 : 플러스
2341 of 2344 : 려니
2342 of 2344 : 뚜껑
2343 of 2344 : 아떻
saving matrix_df!


In [41]:
matrix_df = pd.read_csv('post-data/matrix.csv',low_memory=False)
matrix_df.drop(['Unnamed: 0'], axis=1, inplace=True)
matrix_df

,생후,동산,수도,지남,량,오분,산부인과,확인,마더,노답,...,미달,빼기,막국수,벌써,절개,경제,플러스,려니,뚜껑,아떻
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13793,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
matrix_df = matrix_df.transpose()
matrix_df

,0,1,2,3,4,5,6,7,8,9,...,13785,13786,13787,13788,13789,13790,13791,13792,13793,13794
생후,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
동산,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
수도,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
지남,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
량,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
경제,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
플러스,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
려니,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
뚜껑,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
matrix_df['freq'] = matrix_df.sum(axis=1)
matrix_df = matrix_df[['freq']]
matrix_df

,freq
생후,466
동산,2
수도,36
지남,1
량,11
...,...
경제,1
플러스,2
려니,4
뚜껑,1


In [44]:
freq_df = matrix_df.sort_values('freq' ,ascending=False)
freq_df

print("saving freq_df!")
matrix_df.to_csv("post-data/freq.csv", mode='w', encoding='euc-kr')

saving freq_df!


In [45]:
freq_df

,freq
아기,2987
분유,2457
수유,1649
개월,1314
모유,1255
...,...
욧,1
압타미,1
수포,1
켜,1
